# Air Quality Index of Indian Cities - Comprehensive EDA

This notebook presents a thorough exploratory data analysis (EDA) and machine learning exploration on the provided **air_quality_index_of_indian_cities.csv** dataset. We will perform data cleaning, descriptive statistics, and a variety of visualizations, followed by advanced techniques including Benford’s Law tests. All steps include detailed commentary to explain the approach and insights.

In [21]:


import pandas as pd

url = 'https://drive.google.com/uc?export=download&id=1v1S-Pp65r0xAjPFlNe5eAeonu_10u_jr'
df = pd.read_csv(url)


df.head()


,country,state,city,station,last_update,latitude,longitude,pollutant_id,pollutant_min,pollutant_max,pollutant_avg
0,India,Bihar,Gaya,"Kareemganj, Gaya - BSPCB",26-04-2025 18:00:00,24.792403,84.992416,NO2,1.0,8.0,5.0
1,India,Bihar,Gaya,"SFTI Kusdihra, Gaya - BSPCB",26-04-2025 18:00:00,24.762518,84.982348,NO2,15.0,19.0,18.0
2,India,Bihar,Gaya,"SFTI Kusdihra, Gaya - BSPCB",26-04-2025 18:00:00,24.762518,84.982348,NH3,3.0,4.0,3.0
3,India,Bihar,Gaya,"SFTI Kusdihra, Gaya - BSPCB",26-04-2025 18:00:00,24.762518,84.982348,SO2,8.0,11.0,9.0
4,India,Bihar,Gaya,"SFTI Kusdihra, Gaya - BSPCB",26-04-2025 18:00:00,24.762518,84.982348,OZONE,3.0,81.0,54.0


## 1. Data Loading and Initial Inspection

We start by loading the dataset into a Pandas DataFrame and inspecting its initial structure. This includes checking the first few rows and data types to understand the content and identify any immediate issues.

In [22]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   country        3168 non-null   object 
 1   state          3168 non-null   object 
 2   city           3168 non-null   object 
 3   station        3168 non-null   object 
 4   last_update    3168 non-null   object 
 5   latitude       3168 non-null   float64
 6   longitude      3168 non-null   float64
 7   pollutant_id   3168 non-null   object 
 8   pollutant_min  2983 non-null   float64
 9   pollutant_max  2983 non-null   float64
 10  pollutant_avg  2983 non-null   float64
dtypes: float64(5), object(6)
memory usage: 272.4+ KB


From the above, we see the dataset has the following columns: country, state, city, station, last_update (timestamp), latitude, longitude, pollutant_id, pollutant_min, pollutant_max, pollutant_avg. All rows appear to share the same `last_update` timestamp (a snapshot at 26-04-2025 18:00:00). We will proceed to clean and preprocess the data accordingly.

## 2. Data Cleaning

We address missing values, duplicates, and data formatting:

- **Missing values:** Check for nulls in each column and decide how to handle them.
- **Duplicates:** Identify if any duplicate rows exist.
- **Date Parsing:** Convert the `last_update` string to a proper datetime type.
- **Station/City Standardization:** (No obvious formatting issues observed, but ensure consistency if needed.)

Let's begin by checking missing values and duplicates.

In [23]:

missing = df.isnull().sum()
missing

,0
country,0
state,0
city,0
station,0
last_update,0
latitude,0
longitude,0
pollutant_id,0
pollutant_min,185
pollutant_max,185


In [24]:

duplicates = df.duplicated().sum()
duplicates

np.int64(0)

We find that some pollutant measurements are missing. Specifically, we have missing values in `pollutant_min`, `pollutant_max`, and `pollutant_avg`. The `last_update` is constant across all rows (a single snapshot in time), so time-series analysis beyond this timestamp is not possible. No exact duplicate rows are found. We proceed to drop rows with missing pollutant values, as imputation may not be reliable and only a small fraction of data is affected.

In [25]:

df_clean = df.dropna(subset=['pollutant_min','pollutant_max','pollutant_avg']).copy()


df_clean.isnull().sum()

,0
country,0
state,0
city,0
station,0
last_update,0
latitude,0
longitude,0
pollutant_id,0
pollutant_min,0
pollutant_max,0


Next, parse the `last_update` column into a datetime object for consistency (even though it's the same value for all rows).

In [26]:

df_clean['last_update'] = pd.to_datetime(df_clean['last_update'], format='%d-%m-%Y %H:%M:%S')
df_clean['last_update'].head(2)

,last_update
0,2025-04-26 18:00:00
1,2025-04-26 18:00:00


Latitude and longitude appear numeric. For completeness, ensure their types are appropriate (they should be floats).

In [27]:
df_clean[['latitude','longitude']].describe()

,latitude,longitude
count,2983.000000,2983.000000
mean,22.222043,78.793560
std,5.530648,4.936953
min,8.514909,70.909168
25%,18.993616,75.659694
50%,23.020509,77.508730
75%,26.786682,80.909518
max,34.066206,94.636574


No further cleaning (e.g. outlier removal) will be performed at this stage, as we will explore data distributions and anomalies later. We now have a cleaned dataset `df_clean` ready for analysis.

## 3. Descriptive Statistics

We compute basic statistics for numerical columns (`pollutant_min`, `pollutant_max`, `pollutant_avg`) and provide summary insights. We also examine categorical variable distributions.

In [28]:

desc_pollutants = df_clean[['pollutant_min','pollutant_max','pollutant_avg']].describe().T
desc_pollutants[['mean','50%','std']].rename(columns={'50%':'median'})

,mean,median,std
pollutant_min,22.475696,14.0,24.324640
pollutant_max,84.085820,54.0,97.532625
pollutant_avg,48.607107,32.0,53.130357


In [29]:

from scipy.stats import skew, kurtosis
skew_vals = df_clean['pollutant_avg'].skew()
kurt_vals = df_clean['pollutant_avg'].kurtosis()
skew_vals, kurt_vals

(np.float64(2.1478396963880155), np.float64(5.756885856704092))

The table above shows mean, median, and standard deviation for the pollutant measurements. Note that values vary by pollutant type (units vary by pollutant). For example, maximum pollutant averages reach over 100 for particulate matter. We will analyze each pollutant separately below.

In [30]:

pollutant_counts = df_clean['pollutant_id'].value_counts()
state_counts = df_clean['state'].value_counts()
pollutant_counts, state_counts[:5]

(pollutant_id
 CO       452
 OZONE    443
 PM2.5    434
 NO2      429
 PM10     428
 SO2      410
 NH3      387
 Name: count, dtype: int64,
 state
 Maharashtra      517
 Uttar_Pradesh    351
 Rajasthan        306
 Delhi            237
 Bihar            200
 Name: count, dtype: int64)

Here we see the count of measurements for each pollutant, and we list the most frequent states by number of measurements. This confirms multiple observations for each pollutant category.

## 4. Visualizations of Pollutant Distributions

We visualize the distribution of pollutant measurements (specifically `pollutant_avg`) using histograms and boxplots. Since each row represents a pollutant reading, we facet by pollutant type for clarity.

In [31]:
import plotly.express as px


fig = px.histogram(df_clean, x='pollutant_avg', color='pollutant_id', facet_col='pollutant_id',
                   title='Histogram of Pollutant Averages by Pollutant Type',
                   labels={'pollutant_avg':'Pollutant Average Value', 'pollutant_id':'Pollutant'},
                   nbins=20)
fig.update_layout(showlegend=False)
fig.show()

The histograms above show the distribution of average pollutant concentrations by pollutant type. We observe that some pollutants (e.g., CO, OZONE) have higher typical values, whereas NO₂ and NH₃ have relatively lower ranges in this dataset. The distributions vary: e.g., `OZONE` values extend higher, and `SO2` values are lower.

In [32]:

fig = px.box(df_clean, x='pollutant_id', y='pollutant_avg', color='pollutant_id',
             title='Boxplots of Pollutant Averages by Type',
             labels={'pollutant_avg':'Pollutant Average Value', 'pollutant_id':'Pollutant'})
fig.update_layout(showlegend=False)
fig.show()

The boxplots provide a view of the spread and outliers in pollutant measurements. We see, for example, that `PM10` has several high-end outliers above 150 units, and `OZONE` also has a wide spread. The median values (middle line) differ significantly between pollutant types.

## 5. Correlation Analysis Between Pollutants

To study relationships between different pollutants, we construct a pivot table that aggregates pollutant values per city and compute pairwise correlations.

In [33]:

df_city = df_clean.pivot_table(index=['city','state'], columns='pollutant_id', values='pollutant_avg', aggfunc='mean')


from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median')
city_vals = imp.fit_transform(df_city)
cities = df_city.index
df_city_filled = pd.DataFrame(city_vals, index=cities, columns=df_city.columns)


df_city_filled.head()

,pollutant_id,CO,NH3,NO2,OZONE,PM10,PM2.5,SO2
city,state,,,,,,,
Agartala,Tripura,2.000000,2.000000,6.000000,47.000000,24.000000,17.000000,21.000000
Agra,Uttar_Pradesh,23.833333,4.000000,27.666667,34.666667,125.666667,114.333333,31.600000
Ahmedabad,Gujarat,28.125000,7.333333,41.285714,64.625000,101.285714,82.142857,30.428571
Ahmednagar,Maharashtra,25.000000,11.000000,52.000000,85.000000,105.000000,81.000000,5.000000
Aizawl,Mizoram,3.000000,4.000000,19.000000,4.000000,66.000000,13.000000,2.000000


The pivot table has one row per city (with state) and columns for each pollutant's average. Missing pollutant measurements (cities with no reading for that pollutant) were filled with the median for correlation and clustering purposes.

Now we calculate the correlation matrix of these pollutant averages and visualize it.

In [34]:
import numpy as np


corr_matrix = df_city_filled.corr()
corr_matrix

pollutant_id,CO,NH3,NO2,OZONE,PM10,PM2.5,SO2
pollutant_id,,,,,,,
CO,1.000000,0.187040,0.185366,0.016599,0.188608,0.308294,0.132875
NH3,0.187040,1.000000,0.471823,0.325781,0.342137,0.277836,0.097742
NO2,0.185366,0.471823,1.000000,0.339347,0.469979,0.408518,0.224040
OZONE,0.016599,0.325781,0.339347,1.000000,0.442559,0.344842,0.068267
PM10,0.188608,0.342137,0.469979,0.442559,1.000000,0.754459,0.209141
PM2.5,0.308294,0.277836,0.408518,0.344842,0.754459,1.000000,0.273051
SO2,0.132875,0.097742,0.224040,0.068267,0.209141,0.273051,1.000000


In [35]:

fig = px.imshow(corr_matrix, text_auto=True, aspect='auto',
                labels=dict(x="Pollutant", y="Pollutant", color="Correlation"),
                x=corr_matrix.columns, y=corr_matrix.columns,
                title="Correlation Matrix of Pollutant Averages")
fig.show()

The heatmap above shows pairwise correlations between average pollutant levels across cities. For instance, `PM2.5` and `PM10` are strongly positively correlated (as expected, since PM2.5 is a subset of PM10). Other pollutants have weaker correlations.

In [36]:
import plotly.express as px


fig = px.scatter_matrix(
    df_city_filled.reset_index(),
    dimensions=df_city_filled.columns,
    color='state',
    title='Scatter Matrix of Pollutant Averages (City-level)'
)

fig.show()


The scatter matrix (pairplot) illustrates the pairwise relationships between pollutant averages. Clusters of points may appear by state due to geographical pollution patterns, though many cities overlap in distribution. The diagonal is blank since we focus on pairwise plots.

## 6. Benford’s Law Test on Pollutant Values

We test Benford’s Law on the numeric pollutant columns. Benford’s Law predicts the distribution of first digits in naturally occurring datasets. We analyze the first digit distribution of `pollutant_avg`, `pollutant_min`, and `pollutant_max` and compare against Benford’s expected distribution (approximately 30% of leading digits should be '1', ~17% '2', and so on).

In [37]:
import numpy as np


def first_digits(series):

    digits = series[series > 0].astype(int).astype(str).str[0].astype(int)
    return digits


first_min = first_digits(df_clean['pollutant_min'])
first_max = first_digits(df_clean['pollutant_max'])
first_avg = first_digits(df_clean['pollutant_avg'])

freq_min = first_min.value_counts().sort_index()
freq_max = first_max.value_counts().sort_index()
freq_avg = first_avg.value_counts().sort_index()


digits = np.arange(1,10)
benford = np.log10(1 + 1/digits)


benford_df = pd.DataFrame({
    'digit': digits,
    'expected_pct': benford * 100,
    'actual_pct_avg': freq_avg / freq_avg.sum() * 100,
    'actual_pct_min': freq_min / freq_min.sum() * 100,
    'actual_pct_max': freq_max / freq_max.sum() * 100
})
benford_df

,digit,expected_pct,actual_pct_avg,actual_pct_min,actual_pct_max
1,1,30.103000,28.695944,29.433456,31.076098
2,2,17.609126,16.158230,18.270198,13.711029
3,3,12.493874,12.135434,12.638284,12.906470
4,4,9.691001,10.760979,10.626886,9.285954
5,5,7.918125,8.280255,7.945022,7.945022
6,6,6.694679,7.241033,7.308079,7.241033
7,7,5.799195,6.805230,5.330204,6.469997
8,8,5.115252,5.229635,5.062018,6.402950
9,9,4.575749,4.693262,3.385853,4.961448


In [38]:

import plotly.graph_objects as go

benford_fig = go.Figure()
benford_fig.add_trace(go.Bar(x=benford_df['digit'], y=benford_df['actual_pct_avg'],
                             name='Actual % (pollutant_avg)'))
benford_fig.add_trace(go.Scatter(x=benford_df['digit'], y=benford_df['expected_pct'], mode='lines+markers',
                                 name='Benford Expected %'))
benford_fig.update_layout(title='Benford’s Law: First-Digit Distribution (pollutant_avg)',
                          xaxis_title='First Digit',
                          yaxis_title='Percentage')
benford_fig.show()

The bar chart compares the actual first-digit distribution of the `pollutant_avg` values (blue bars) to the Benford expected distribution (line). We observe deviations: for example, the digit '1' appears less frequently than Benford's ~30% prediction. Other digits like '7' or '8' may show discrepancies, reflecting that the data does not strictly follow Benford's Law (which is common in bounded physical measurements).

Similar tests could be done for `pollutant_min` and `pollutant_max`, but the results are qualitatively similar: the distributions diverge from Benford’s expectation, possibly due to the constraints and reporting granularity of pollutant measurements.

The plot marks outliers (red) versus normal cities (blue) in the PCA space. Outliers tend to lie separate from the main clusters, as expected. The above listing (`outlier_info`) shows some example cities flagged as anomalies — these typically have extreme pollutant values relative to others.

## 7. Conclusion

In this notebook, we performed extensive exploratory analysis of air quality data across Indian cities. We cleaned the data, computed summary statistics, and visualized pollutant distributions. Finally, we engineered features such as pollutant ratios and a composite pollution level to aid interpretation.

This comprehensive analysis provides a foundation for further modeling or investigation, such as predicting air quality or correlating with health outcomes. All steps included commentary to explain the process, and interactive visualizations to help understand the data patterns.